In [75]:
import datetime
import time
import requests  
import csv
import sys

PROMETHEUS = '' # Fill your settings

def query_api(query):
    response =requests.get(PROMETHEUS + '/api/v1/query', params={'query': query}) 
    results = response.json()['data']['result']
    return results

def write_csv(filename, prom_query_results, renameFields={'__name__': 'metric'}, dropFields=[]):
    file = open(filename, 'w+')
    writer = csv.writer(file)
    labelnames = set()
    for result in prom_query_results:
        labelnames.update(result['metric'].keys())
    for drop in dropFields:
        labelnames.discard(drop)
    
    labelnames = sorted(labelnames)

    writer.writerow([name if name not in renameFields else renameFields[name] for name in labelnames] + ['value'] if 'value' not in dropFields else [])
    
    for result in prom_query_results:
        l = []
        for label in labelnames:
            l.append(result['metric'].get(label, ''))
        if 'value' not in dropFields:
            l.append(result['value'][1])
        writer.writerow(l)
        break
    file.close()
    return


results = query_api('group by(job, __name__) ({__name__!=""})')
for result in results:
    obj = result['metric']
    metric = obj['__name__'] if '__name__' in obj else None
    job = obj['job'] if 'job' in obj else None
    prom_filter = '' if job is None else '{job="%s"}' % job
    query = metric + prom_filter
    query_result = query_api(query)
    
    filename = ''
    if metric:
        filename += metric
    if job:
        if len(filename) > 0:
            filename += '_'
        filename += job
    filename += '.csv'
    write_csv(filename, query_result)



